# Intel(R) Media SDK Utilisation in Video Applications

In previous labs, we have focuced on DL inference on the video applications but a lot more going on a video analytics applications. Intel provides a lot more tool to enhace the process.

At this section, we will run 6 application developed with OpenCV, Inference Engine and Media SDK C++ APIs each named as `tutorial_0` to `tutorial_5`.

All the source code of these examples placed under `/home/intel/Tutorials/interop_tutorials` folder.

|App Name   | Decoding   | Pre-Process | Inference   | Post-Process | Encoding  |
|-----------|------------|-------------|-------------|--------------|-----------|
|Tutorial 0 | OpenCV     |     OpenCV  | OpenCV      | OpenCV       | OpenCV    |
|Tutorial 1 | OpenCV     |     OpenCV  | OpenVINO    | OpenCV       | OpenCV    |
|Tutorial 2 | Media SDK  |     OpenCV  | OpenVINO    | OpenCV       | OpenCV    |
|Tutorial 3 | Media SDK  |  Media SDK  | OpenVINO    | OpenCV       | OpenCV    |
|Tutorial 4 | Media SDK  |  Media SDK  | OpenVINO    | Media SDK    | OpenCV    |
|Tutorial 5 | Media SDK  |  Media SDK  | OpenVINO    | Media SDK    | Media SDK |

In this set of tutorails, we will investigate how Intel(R) Media SDK into an End to End Video Application.

All applications uses `SSD GoogleNet v2` for object recognition this time. 

# Tutorial 1

Tutorial 1 does all decoding, pre and post-processing part with OpenCV* and inference with Intel(R) Distribution of OpenVINO(TM).

Let's try out Tutorial 1 sample application.

Open a new terminal. (ctrl+alt+t)

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

cd /home/intel/Tutorials/interop_tutorials/tutorial_1

./tutorial1 -h

[usage]
	tutorial_1 [option]
	options:

		-h                  Print a usage message
		-i <path/filename>  Required. Path to input video file
		-fr <val>           Number of frames from stream to process
		-m <path/filename>  Required. Path to IR .xml file.
		-l <path/filename>  Required. Path to labels file.
		-d <device>         Infer target device (CPU or GPU or MYRIAD)
		-pc                 Enables per-layer performance report
		-thresh <val>       Confidence threshold for bounding boxes 0-1
		-batch <val>        Batch size
		-s                  Display less information on the screen
		-e <path/filename>  Load layer extension plugin
		-mean               Mean value for normalization of data during planar BGR blob preprocess step
		-scale              Scale value for normalization of data during planar BGR blob preprocess step
		-show               Display a window for composited image with ROI

```

As seen from above example, this application is able to load OpenVINO model and is able to run inference on CPU, GPU and MYRIAD. 

We will use `run.sh` script to run `tutorial1` on CPU and `run_gpu.sh` to run inference on GPU. You can change commands in scripts as you wish. 

```bash

./run.sh

> Pre-stage average:	10.00 ms/frame (decoding, color converting, resizing)
> Infer average:	55.19 ms/frame (inferencing)
> Post-stage average:	0.65 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 17.12 sec

Done!
```

Let's run on GPU, this time IE loads clDNNPlugin. 

```bash

./run_gpu.sh

> Pre-stage average:	10.41 ms/frame (decoding, color converting, resizing)
> Infer average:	27.67 ms/frame (inferencing)
> Post-stage average:	0.71 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 10.01 sec

Done!

```

Final output of inference written to out.h264 file, you can play it with mplayer and see the inference output.

```bash
mplayer out.h264`
```

Let's get into details and investigate the perofmance little more.

## Performance Analysis

As we seen from the previous sections. We are able to get performance analysis. It is also implemented for this application. It will be a messy output to see each inference performance outputs.

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_1/tutorial1 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -pc
```

## Output Video

Let's see the output interactively with `-show` option.

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_1/tutorial1 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -show
```

## Batch Support

Let's see how we can use a larger batch size with this example.

At the output, you should realize that, it does show 32 inferences which means it packed 256 frames to batches and does inference at the all 8 same time, and we gain a lot of time, total execution decreased a lot. 

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_1/tutorial1 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -batch 8

Batch: 29/32
	pre-stage:	5.84 ms/frame
	infer:		21.98 ms/frame
	post-stage:	5.39 ms/frame

Batch: 30/32
	pre-stage:	6.43 ms/frame
	infer:		22.82 ms/frame
	post-stage:	5.38 ms/frame

Batch: 31/32
	pre-stage:	5.99 ms/frame
	infer:		22.39 ms/frame
	post-stage:	4.69 ms/frame



> Pre-stage average:	8.98 ms/frame (decoding, color converting, resizing)
> Infer average:	22.18 ms/frame (inferencing)
> Post-stage average:	5.64 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 8.62 sec

Done!

```

## Tutorial 2

In Tutorial 2 application, we add Media SDK for decoding only, and Media SDK uses system memory at this stage. In this tutorial and the followings, we don't have display but they all saving the output of the video. 


```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

cd /home/intel/Tutorials/interop_tutorials/tutorial_2

./tutorial2 -h

[usage]
	tutorial_2 [option]
	options:

		-h                  Print a usage message
		-i <path/filename>  Required. Path to input video file, video elementary stream only
		-fr <val>           Number of frames from stream to process
		-m <path/filename>  Required. Path to IR .xml file.
		-l <path/filename>  Required. Path to labels file.
		-d <device>         Infer target device (CPU or GPU or MYRIAD)
		-pc                 Enables per-layer performance report
		-thresh <val>       Confidence threshold for bounding boxes 0-1
		-batch <val>        Batch size
		-s                  Display less information on the screen
		-e <path/filename>  Load layer extension plugin
		-mean               Mean value for normalization of data during planar BGR blob preprocess step
		-scale              Scale value for normalization of data during planar BGR blob preprocess step

```

As seen from above example, this application is able to load OpenVINO model and is able to run inference on CPU, GPU and MYRIAD. 

We will use run.sh file to run tutorial2 on CPU and run_gpu.sh to run inference on GPU.

We see that there is a particular improvement on inference. 

```bash

./run.sh

> Pre-stage average:	11.92 ms/frame (decoding, color converting, resizing)
> Infer average:	55.72 ms/frame (inferencing)
> Post-stage average:	0.50 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 17.62 sec

> Done ! (Output is in out.h264 -> $ mplayer out.h264)

```

Let's run on GPU, this time IE loads clDNNPlugin. Now, you will realized that there isn't any improvement on the performance. Why? 

```bash

./run_gpu.sh

> Pre-stage average:	10.93 ms/frame (decoding, color converting, resizing)
> Infer average:	40.24 ms/frame (inferencing)
> Post-stage average:	0.62 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 13.32 sec

> Done ! (Output is in out.h264 -> $ mplayer out.h264)

```

Final output of inference written to out.h264 file, you can play it with mplayer and see the inference output.

```bash
mplayer out.h264`
```

Let's get into details and investigate the perofmance little more.

## Performance Analysis

As we seen from the previous sections. We are able to get performance analysis. It is also implemented for this application. It will be a messy output to see each inference performance outputs.

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_2/tutorial2 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -pc
```

## Batch Support

Let's see how we can use a larger batch size with this example.

At the output, you should realize that, it does show 32 inferences which means it packed 256 frames to batches and does inference at the all 8 same time. 

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_2/tutorial2 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -batch 8

Batch: 28/32
	pre-stage:	6.44 ms/frame
	infer:		22.91 ms/frame
	post-stage:	1.61 ms/frame

Batch: 29/32
	pre-stage:	6.78 ms/frame
	infer:		22.73 ms/frame
	post-stage:	2.14 ms/frame

Batch: 30/32
	pre-stage:	6.56 ms/frame
	infer:		22.39 ms/frame
	post-stage:	1.61 ms/frame



> Pre-stage average:	10.44 ms/frame (decoding, color converting, resizing)
> Infer average:	22.62 ms/frame (inferencing)
> Post-stage average:	2.13 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 9.20 sec

> Done ! (Output is in out.h264 -> $ mplayer out.h264)

```

# Tutorial 3

In Tutorial 3 application, we add Media SDK for decoding and pre-processing let's see difference on the pre-stage.

Let's open a new terminal. 

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

cd /home/intel/Tutorials/interop_tutorials/tutorial_3

./tutorial3 -h

[usage]
	tutorial_3 [option]
	options:

		-h                  Print a usage message
		-i <path/filename>  Required. Path to input video file, video elementary stream only
		-fr <val>           Number of frames from stream to process
		-m <path/filename>  Required. Path to IR .xml file.
		-l <path/filename>  Required. Path to labels file.
		-d <device>         Infer target device (CPU or GPU or MYRIAD)
		-pc                 Enables per-layer performance report
		-thresh <val>       Confidence threshold for bounding boxes 0-1
		-batch <val>            Batch size
		-s                  Display less information on the screen
		-e <path/filename>  Load layer extension plugin
		-mean               Mean value for normalization of data during planar BGR blob preprocess step
		-scale              Scale value for normalization of data during planar BGR blob preprocess step

```

As seen from above example, this application is able to load OpenVINO model and is able to run inference on CPU, GPU and MYRIAD. 

We will use run.sh file to run tutorial3 on CPU and run_gpu.sh to run inference on GPU.

We see that there is a particular improvement on inference. 

```bash

./run.sh

> Pre-stage average:	7.97 ms/frame (decoding, color converting, resizing)
> Infer average:	55.23 ms/frame (inferencing)
> Post-stage average:	0.56 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 16.45 sec

> Done ! (Output is in out.h264 -> $ mplayer out.h264)

```

Let's run on GPU, this time IE loads clDNNPlugin. 

```bash

./run_gpu.sh

> Pre-stage average:	8.89 ms/frame (decoding, color converting, resizing)
> Infer average:	40.29 ms/frame (inferencing)
> Post-stage average:	0.75 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 12.85 sec

> Done ! (Output is in out.h264 -> $ mplayer out.h264)

```

Final output of inference written to out.h264 file, you can play it with mplayer and see the inference output.

```bash
mplayer out.h264
```

Let's get into details and investigate the perofmance little more.

## Performance Analysis

As we seen from the previous sections. We are able to get performance analysis. It is also implemented for this application. It will be a messy output to see each inference performance outputs.

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_3/tutorial3 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -pc
```

## Batch Support

Let's see how we can use a larger batch size with this example.

At the output, you should realize that, it does show 32 inferences which means it packed 256 frames to batches and does inference at the all 8 same time. 

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_3/tutorial3 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -batch 8

Batch: 30/32
	pre-stage:	8.21 ms/frame
	infer:		21.79 ms/frame
	post-stage:	2.25 ms/frame

Batch: 31/32
	pre-stage:	9.61 ms/frame
	infer:		22.20 ms/frame
	post-stage:	2.32 ms/frame

Batch: 32/32
	pre-stage:	8.91 ms/frame
	infer:		22.20 ms/frame
	post-stage:	3.41 ms/frame

> Pre-stage average:	8.33 ms/frame (decoding, color converting, resizing)
> Infer average:	22.43 ms/frame (inferencing)
> Post-stage average:	2.43 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 8.54 sec

> Done ! (Output is in out.h264 -> $ mplayer out.h264)

```

# Tutorial 4

In Tutorial 4 application, we add Media SDK for decoding, pre-processing and post-processing steps let's see difference on the pre-stage. Difference on Tutorial 4 is that, Media SDK uses Video Memory instead of System Memory.

Let's open a new terminal. 

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

cd /home/intel/Tutorials/interop_tutorials/tutorial_4

./tutorial4 -h

[usage]
	tutorial_4 [option]
	options:

		-h                  Print a usage message
		-i <path/filename>  Required. Path to input video file, video elementary stream only
		-fr <val>           Number of frames from stream to process
		-m <path/filename>  Required. Path to IR .xml file.
		-l <path/filename>  Required. Path to labels file.
		-d <device>         Infer target device (CPU or GPU or MYRIAD)
		-pc                 Enables per-layer performance report
		-thresh <val>       Confidence threshold for bounding boxes 0-1
		-batch <val>        Batch size
		-s                  Display less information on the screen
		-e <path/filename>  Load layer extension plugin
		-mean               Mean value for normalization of data during planar BGR blob preprocess step
		-scale              Scale value for normalization of data during planar BGR blob preprocess step

```

As seen from above example, this application is able to load OpenVINO model and is able to run inference on CPU, GPU and MYRIAD. 

We will use run.sh file to run tutorial4 on CPU and run_gpu.sh to run inference on GPU.

We see that there is a particular improvement on inference. 

```bash

./run.sh

> Pre-stage average:	5.01 ms/frame (decoding, color converting, resizing)
> Infer average:	55.29 ms/frame (inferencing)
> Post-stage average:	0.52 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 15.71 sec

> Done ! (Output is in out.h264 -> $ mplayer out.h264)

```

Let's run on GPU, this time IE loads clDNNPlugin. 

```bash

./run_gpu.sh

> Pre-stage average:	4.95 ms/frame (decoding, color converting, resizing)
> Infer average:	42.14 ms/frame (inferencing)
> Post-stage average:	0.54 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 12.28 sec

> Done ! (Output is in out.h264 -> $ mplayer out.h264)

```

Final output of inference written to out.h264 file, you can play it with mplayer and see the inference output.

```bash
mplayer out.h264
```

Let's get into details and investigate the perofmance little more.

## Performance Analysis

As we seen from the previous sections. We are able to get performance analysis. It is also implemented for this application. It will be a messy output to see each inference performance outputs.

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_4/tutorial4 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -pc
```

## Batch Support

Let's see how we can use a larger batch size with this example.

At the output, you should realize that, it does show 32 inferences which means it packed 256 frames to batches and does inference at the all 8 same time. 

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_4/tutorial4 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -batch 8

Batch: 30/32
	pre-stage:	4.87 ms/frame
	infer:		22.76 ms/frame
	post-stage:	2.34 ms/frame

Batch: 31/32
	pre-stage:	4.74 ms/frame
	infer:		22.19 ms/frame
	post-stage:	2.23 ms/frame

Batch: 32/32
	pre-stage:	5.00 ms/frame
	infer:		22.05 ms/frame
	post-stage:	2.28 ms/frame


> Pre-stage average:	4.97 ms/frame (decoding, color converting, resizing)
> Infer average:	22.03 ms/frame (inferencing)
> Post-stage average:	1.91 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 7.51 sec

> Done ! (Output is in out.h264 -> $ mplayer out.h264)

```

# Tutorial 5

In Tutorial 5 application, we add Media SDK for decoding, pre-processing, post-processing and encoding steps let's see difference on the pre-stage.

Let's open a new terminal. 

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

cd /home/intel/Tutorials/interop_tutorials/tutorial_5

./tutorial5 -h

[usage]
	tutorial_5 [option]
	options:

		-h                  Print a usage message
		-i <path/filename>  Required. Path to input video file, video elementary stream only
		-fr <val>           Number of frames from stream to process
		-m <path/filename>  Required. Path to IR .xml file.
		-l <path/filename>  Required. Path to labels file.
		-d <device>         Infer target device (CPU or GPU or MYRIAD)
		-pc                 Enables per-layer performance report
		-thresh <val>       Confidence threshold for bounding boxes 0-1
		-b <val>            Batch size
		-s                  Display less information on the screen
		-e <path/filename>  Load layer extension plugin
		-mean               Mean value for normalization of data during planar BGR blob preprocess step
		-scale              Scale value for normalization of data during planar BGR blob preprocess step


```

As seen from above example, this application is able to load OpenVINO model and is able to run inference on CPU, GPU and MYRIAD. 

Let's run on GPU, this time IE loads clDNNPlugin. 

```bash

./run.sh

> Pre-stage average:	4.82 ms/frame (decoding, color converting, resizing)
> Infer average:	55.69 ms/frame (inferencing)
> Post-stage average:	2.61 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 16.24 sec

```

```bash

./run_gpu.sh


> Pre-stage average:	4.89 ms/frame (decoding, color converting, resizing)
> Infer average:	40.73 ms/frame (inferencing)
> Post-stage average:	2.75 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 12.43 sec

Done!

```

Final output of inference written to out.h264 file, you can play it with mplayer and see the inference output.

```bash
mplayer out.h264
```

Let's get into details and investigate the perofmance little more.

## Performance Analysis

As we seen from the previous sections. We are able to get performance analysis. It is also implemented for this application. It will be a messy output to see each inference performance outputs.

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_5/tutorial5 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -pc
```

## Batch Support

Let's see how we can use a larger batch size with this example.

At the output, you should realize that, it does show 32 inferences which means it packed 256 frames to batches and does inference at the all 8 same time. 

```bash
source /opt/intel/computer_vision_sdk/bin/setupvars.sh
export LD_LIBRARY_PATH=/home/intel/inference_engine_samples/intel64/Release/lib:$LD_LIBRARY_PATH

/home/intel/Tutorials/interop_tutorials/tutorial_5/tutorial5 -m /home/intel/Tutorials/test_content/IR/SSD/SSD_GoogleNet_v2_fp16.xml -i /home/intel/Tutorials/test_content/video/cars_1920x1080.h264 -l /home/intel/Tutorials/test_content/IR/SSD/pascal_voc_classes.txt -d GPU -batch 8

Batch: 31/32
	pre-stage:	7.95 ms/frame
	infer:		22.76 ms/frame
	post-stage:	5.50 ms/frame

Batch: 32/32
	pre-stage:	8.23 ms/frame
	infer:		22.83 ms/frame
	post-stage:	5.95 ms/frame

Batch: 33/32
	pre-stage:	6.54 ms/frame
    

> Pre-stage average:	5.58 ms/frame (decoding, color converting, resizing)
> Infer average:	22.83 ms/frame (inferencing)
> Post-stage average:	7.73 ms/frame (drawing bounding box, encoding, saving)

> Total elapsed execution time: 9.32 sec


Done!

```